목적 : 비상장 외감기업 탐색적 데이터 분석 EDA
=====================

목표 : 기업수명주기별, 부실별, 기초통계량, 정규성, Q-Qplot
-------------------------    

import modules

In [102]:
import pandas as pd
import numpy as np
from collections import Counter

%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
import statsmodels.api as sm

import platform
if platform.system() == "Darwin":           # MAC인 경우
    plt.rc('font', family ="AppleGothic")
else:
    plt.rc('font',family='Malgun Gothic')

#멧플로립에서 음수를 표시
plt.rc('axes',unicode_minus=False)

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')


import scipy.stats as stats
from scipy.stats import shapiro, pearsonr
from itertools import combinations
from sklearn.model_selection import train_test_split

환경설정

In [103]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# pd.set_option('display.max_columns', None)
pd.reset_option('display.max_columns')

pd.set_option('display.max_seq_items', None)

pd.set_option('float_format', '{:.3f}'.format) # 소수점 3개까지 출력

load file

In [104]:
unlisted = pd.read_csv("../data/datasets/비상장수정필요피처.csv")
# 영업년수와 상장년수가 있더라

In [105]:
unlisted.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

# train test split

In [106]:
X = unlisted.drop('부실판단', axis=1)
y = unlisted['부실판단']
# 데이터를 학습용과 테스트용으로 나눔
unlisted_X_train, unlisted_X_test, unlisted_y_train, unlisted_y_test = train_test_split(X, y, test_size= 1 / (np.exp(1) + 1) , random_state=911, stratify=y)

In [107]:
unlisted_X_train['부실판단'] = unlisted_y_train

In [108]:
len(X)

20707

In [109]:
len(unlisted_X_train)

15138

In [110]:
len(unlisted_X_test)

5569

# describe

all

In [111]:
unlisted_X_train

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
7716,(주)동남코리아,70118,2022,-18.740,-39.580,-3.410,-27.330,26.090,17.940,22.200,...,0.414,-1.063,-0.394,76.854,-1.323,1,0,0,0,0.000
4739,피에스케이홀딩스(주),20666,2019,130.010,22131.400,10.600,36.690,1385.060,1230.540,8.880,...,0.289,2.053,0.503,-1.195,-3.927,0,1,0,0,0.000
12685,금용해양산업주식회사,63939,2022,121.510,368.960,29.460,46.110,159.900,159.900,32.010,...,0.049,1.080,0.999,-2.679,-1.741,0,1,0,0,0.000
17405,주식회사대일,53349,2014,-2.390,-3.210,-1.850,-2.010,87.410,23.480,341.770,...,0.102,2.030,1.585,0.070,-0.142,1,0,0,0,1.000
18617,주식회사삼호프레스,83444,2018,-1.480,-2.030,-0.910,-2.540,71.900,59.910,160.760,...,0.237,6.115,0.447,-0.385,0.462,0,0,0,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13013,농업회사법인한만두식품주식회사,82850,2022,6.720,8.850,5.280,5.950,95.860,63.560,367.770,...,0.172,4.615,1.414,-0.155,0.420,0,0,1,0,0.000
12657,금성냉장주식회사,29681,2014,-0.530,-1.020,-0.440,-0.520,28.600,13.240,484.900,...,0.083,-0.938,0.937,-1.821,-2.265,0,1,0,0,1.000
11925,(주)노비아레텍,70975,2022,-7.670,-11.400,-5.630,-4.340,100.960,38.140,276.910,...,0.069,7.126,1.497,3.183,-0.746,1,0,0,0,1.000
10995,새한에프앤비(주),13761,2014,-9.190,-10.580,-6.360,-17.210,45.060,45.060,224.730,...,1.143,-7.013,1.058,-0.725,2.522,0,0,0,1,1.000


In [112]:
unlisted_X_train.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,...,15125.000,15138.000,15137.000,15138.000,14908.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,67773.011,2019.874,19.314,724.234,3.241,-22.741,431.245,358.475,1378.936,983.416,...,0.504,9.524,0.895,117.567,-5.947,0.218,0.368,0.191,0.223,0.361
std,23619.347,2.797,1261.272,31970.320,37.792,539.239,5347.663,5224.565,35986.322,30885.126,...,2.603,364.613,32.023,30082.690,678.697,0.413,0.482,0.393,0.416,0.480
min,10037.000,2014.000,-86467.650,-323703.660,-3388.620,-21110.050,0.070,0.070,0.000,0.000,...,0.000,-9823.523,-3383.817,-1368867.400,-35585.135,0.000,0.000,0.000,0.000,0.000
25%,52329.000,2018.000,-4.480,-5.120,-2.880,-4.890,56.050,31.620,60.845,36.860,...,0.086,-0.142,0.453,-2.527,-1.760,0.000,0.000,0.000,0.000,0.000
50%,70853.000,2022.000,2.920,1.510,1.860,2.360,108.715,72.700,163.855,102.565,...,0.162,2.091,0.956,-0.802,-0.770,0.000,0.000,0.000,0.000,0.000
75%,87046.750,2022.000,14.328,17.380,7.387,10.777,197.823,150.323,373.438,239.108,...,0.393,4.266,1.352,-0.063,0.216,0.000,1.000,0.000,0.000,1.000
max,101635.000,2022.000,113999.840,3194164.340,1274.750,21579.540,447760.110,447760.110,3385006.150,2905990.690,...,183.355,31473.399,743.445,3001862.889,32320.540,1.000,1.000,1.000,1.000,1.000


In [113]:
unlisted_X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15138 entries, 7716 to 12115
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                15138 non-null  object 
 1   거래소코드              15138 non-null  int64  
 2   회계년도               15138 non-null  int64  
 3   CASH FLOW 대 부채비율   15138 non-null  float64
 4   CASH FLOW 대 차입금비율  15138 non-null  float64
 5   CASH FLOW 대 총자본비율  15138 non-null  float64
 6   CASH FLOW 대 매출액비율  15138 non-null  float64
 7   유동비율               15138 non-null  float64
 8   당좌비율               15138 non-null  float64
 9   부채비율               15138 non-null  float64
 10  유동부채비율             15138 non-null  float64
 11  차입금의존도             15138 non-null  float64
 12  순운전자본비율            15138 non-null  float64
 13  현금비율               15138 non-null  float64
 14  유동자산구성비율           15138 non-null  float64
 15  자기자본구성비율           15138 non-null  float64
 16  자기자본순이익률           15138

In [114]:
unlisted_X_train.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [115]:
unlisted_X_train.isna().sum().sort_values(ascending=False)

영업재무현금비율             230
쭈피처                   13
재무레버리지                 1
회사명                    0
경영자본회전률                0
부가가치(백만원)              0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
유형자산회전율                0
비유동자산회전률               0
총자본회전률                 0
매출액증가율                 0
유동자산회전률                0
영업년수                   0
영업레버리지                 0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
유형자산증가율                0
비유동자산증가율               0
거래소코드                  0
현금비율                   0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0


In [116]:
unlisted_X_train.drop(['부실판단', '도입기', '성장기', '성숙기', '쇠퇴기'], axis=1).isin([0]).sum().sort_values(ascending=False) / len(unlisted_X_train) * 100

자본분배율               10.556
이윤분배율                9.169
총자산회전율               8.944
총자산증가율               8.938
차입금의존도               8.409
CASH FLOW 대 차입금비율    8.376
매출채권회전률              7.802
자기자본순이익률             4.987
자기자본구성비율             3.970
부채비율                 3.957
유동부채비율               3.957
유형자산증가율              2.299
설비투자효율               1.427
유형자산회전율              1.249
매출액증가율               0.958
현금비율                 0.595
금융비용부담률              0.548
총자본회전률               0.357
비유동자산증가율             0.304
경영자본회전률              0.244
비유동자산회전률             0.185
영업년수                 0.106
유동자산회전률              0.092
총자본사업이익률             0.059
총자본증가율               0.053
총자본영업이익률             0.053
CASH FLOW 대 총자본비율    0.053
매출액영업이익률             0.053
CASH FLOW 대 매출액비율    0.053
경영자본순이익률             0.046
CASH FLOW 대 부채비율     0.033
유동자산증가율              0.026
매출액순이익률              0.026
총자본투자효율              0.026
당좌자산회전률              0.013
CROE                 0.013
부가가치율                0.013
순

In [117]:
unlisted_X_train.isin([np.inf]).sum().sort_values(ascending=False)

총자산증가율               1
회사명                  0
매출액증가율               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
유동자산증가율              0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율   

In [118]:
unlisted_X_train.isin([-np.inf]).sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

In [119]:
# inf_col = unlisted_X_train.isin([np.inf]).sum().sort_values(ascending=False).index[0]
# max_value = unlisted_X_train[inf_col].replace(np.inf, np.nan).max()
# unlisted_X_train[inf_col] = unlisted_X_train[inf_col].replace(np.inf, max_value)

# 범주 분할 1현금흐름 / 2안정성 / 2수익성 / 3유동성 / 4성장성 / 5생산성 / 6활동성 / 7시장지표 및 기타

In [120]:
# 비상장 all data는 unlisted
# 비상장 label들

labels = unlisted_X_train[
    [
        '거래소코드',
        '회계년도',
        '부실판단',
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기'
    ]
]

# 비상장 피처
features = unlisted_X_train.drop(
    labels=[
        '회사명',
        '거래소코드',
        '회계년도',        
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'        
    ],
    axis=1,    
)

# 1현금흐름 4개
cash_ft1 = unlisted_X_train[
    [
        'CASH FLOW 대 부채비율',
        'CASH FLOW 대 차입금비율',
        'CASH FLOW 대 총자본비율',
        'CASH FLOW 대 매출액비율',
        '부실판단'
    ]
]

# 2안정성 9개
stability_ft1 = unlisted_X_train[
    [
        '유동비율',
        '당좌비율',
        '부채비율',
        '유동부채비율',
        '차입금의존도',
        '순운전자본비율',
        '현금비율',
        '유동자산구성비율',
        '자기자본구성비율',
        '부실판단'
    ]
]

# 3수익성 8개
profitability_ft1 = unlisted_X_train[
    [
        '자기자본순이익률',
        '경영자본순이익률',
        '매출액순이익률',
        '총자본사업이익률',
        '총자본영업이익률',
        '매출액영업이익률',
        '금융비용부담률',
        'CROE',
        '부실판단'
    ]
]

# 4성장성 6개
growth_ft1 = unlisted_X_train[
    [
        '총자산증가율',
        '총자본증가율',
        '유동자산증가율',
        '비유동자산증가율',
        '매출액증가율',
        '유형자산증가율',
        '부실판단'
    ]
]

# 5생산성 6개
productivity_ft1 = unlisted_X_train[
    [
        '부가가치(백만원)',
        '총자본투자효율',
        '설비투자효율',
        '부가가치율',
        '이윤분배율',
        '자본분배율',
        '부실판단'
    ]
]

# 6활동성 8개
activity_ft1 = unlisted_X_train[
    [
        '총자산회전율',
        '매출채권회전률',
        '당좌자산회전률',
        '유형자산회전율',
        '경영자본회전률',
        '총자본회전률',
        '비유동자산회전률',
        '유동자산회전률',
        '부실판단'
    ]
]

# 7기타
etc_ft1 = unlisted_X_train[
    [
        '영업년수',
        '쭈피처',
        '영업레버리지',
        '재무레버리지',
        '영업투자현금비율',
        '영업재무현금비율', 
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'
    ]
]

In [121]:
unlisted_X_train.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

1현금흐름 4가지 cash

In [122]:
cash_ft1.describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000
mean,19.314,724.234,3.241,-22.741,0.361
std,1261.272,31970.320,37.792,539.239,0.480
min,-86467.650,-323703.660,-3388.620,-21110.050,0.000
25%,-4.480,-5.120,-2.880,-4.890,0.000
50%,2.920,1.510,1.860,2.360,0.000
75%,14.328,17.380,7.387,10.777,1.000
max,113999.840,3194164.340,1274.750,21579.540,1.000


2안정성 9가지 stability

In [123]:
stability_ft1.describe()

,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,431.245,358.475,1378.936,983.416,43.176,0.167,98.637,45.603,38.894,0.361
std,5347.663,5224.565,35986.322,30885.126,211.937,297.974,2442.606,28.781,26.118,0.480
min,0.070,0.070,0.000,0.000,0.000,-36261.200,-0.200,0.000,0.000,0.000
25%,56.050,31.620,60.845,36.860,17.360,-19.517,1.590,21.870,18.270,0.000
50%,108.715,72.700,163.855,102.565,42.040,2.925,8.605,43.180,35.145,0.000
75%,197.823,150.323,373.438,239.108,61.450,26.938,32.528,67.630,57.267,1.000
max,447760.110,447760.110,3385006.150,2905990.690,25856.360,99.690,275285.960,100.000,99.980,1.000


3수익성 8가지 profitability

In [124]:
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,6674.974,30.528,538176.590,46.170,-95.676,8.411,0.361
std,352176.022,1669.910,34787344.705,747.415,2660.033,1074.584,0.480
min,-68167.780,-246.020,-563419210.000,-13763.270,-208392.960,-94627.640,0.000
25%,937.122,4.870,9.130,7.960,-0.310,22.102,0.000
50%,2281.575,11.060,28.820,16.220,11.390,55.870,0.000
75%,4883.170,19.218,97.688,33.188,35.943,74.770,1.000
max,43327598.520,205392.780,4053778655.000,70947.820,8226.360,126.190,1.000


4성장성 6개 growth

In [125]:
growth_ft1.describe()

,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,inf,6.597,216.233,302.142,228.063,192381.296,0.361
std,NaN,55.243,20598.824,11247.279,9908.536,21029765.042,0.480
min,-100.000,-99.890,-99.900,-100.000,-99.910,-100.000,0.000
25%,-4.447,-5.480,-14.060,-5.207,-12.510,-5.630,0.000
50%,0.523,1.820,3.990,-0.110,3.610,-1.040,0.000
75%,11.191,12.805,25.038,12.480,21.797,8.002,1.000
max,inf,5081.220,2531488.340,1154781.210,900275.000,2580453089.000,1.000


5생산성 6가지 productivity

In [126]:
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,6674.974,30.528,538176.590,46.170,-95.676,8.411,0.361
std,352176.022,1669.910,34787344.705,747.415,2660.033,1074.584,0.480
min,-68167.780,-246.020,-563419210.000,-13763.270,-208392.960,-94627.640,0.000
25%,937.122,4.870,9.130,7.960,-0.310,22.102,0.000
50%,2281.575,11.060,28.820,16.220,11.390,55.870,0.000
75%,4883.170,19.218,97.688,33.188,35.943,74.770,1.000
max,43327598.520,205392.780,4053778655.000,70947.820,8226.360,126.190,1.000


6활동성 8가지 activity

In [127]:
activity_ft1.describe()

,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,0.938,700.636,4.246,7686.666,1.119,0.996,22.988,2.830,0.361
std,1.663,46090.572,7.518,263256.703,1.590,1.404,375.217,5.156,0.480
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.162,3.990,1.460,0.650,0.340,0.280,0.490,1.060,0.000
50%,0.652,7.150,2.880,2.060,0.820,0.730,1.460,1.990,0.000
75%,1.231,13.370,4.940,7.050,1.430,1.300,3.757,3.340,1.000
max,100.540,5386048.240,344.060,19823298.290,106.800,92.050,32610.280,344.060,1.000


7기타 etc

In [128]:
etc_ft1.describe()

,영업년수,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,15138.000,15125.000,15138.000,15137.000,15138.000,14908.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,21.322,0.504,9.524,0.895,117.567,-5.947,0.218,0.368,0.191,0.223,0.361
std,11.151,2.603,364.613,32.023,30082.690,678.697,0.413,0.482,0.393,0.416,0.480
min,0.000,0.000,-9823.523,-3383.817,-1368867.400,-35585.135,0.000,0.000,0.000,0.000,0.000
25%,13.000,0.086,-0.142,0.453,-2.527,-1.760,0.000,0.000,0.000,0.000,0.000
50%,20.000,0.162,2.091,0.956,-0.802,-0.770,0.000,0.000,0.000,0.000,0.000
75%,27.000,0.393,4.266,1.352,-0.063,0.216,0.000,1.000,0.000,0.000,1.000
max,110.000,183.355,31473.399,743.445,3001862.889,32320.540,1.000,1.000,1.000,1.000,1.000


# 도입정상

In [129]:
도입정상 = unlisted_X_train[(unlisted_X_train['도입기'] == 1) & (unlisted_X_train['부실판단'] == 0)]

도입정상 결측치확인

In [130]:
도입정상.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

# 도입부실

In [131]:
도입부실 = unlisted_X_train[(unlisted_X_train['도입기'] == 1) & (unlisted_X_train['부실판단'] == 1)]

도입부실 결측치확인

In [132]:
도입부실.isna().sum().sort_values(ascending=False)

쭈피처                  1
회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

정규성 확인

In [133]:
na_col = 도입부실.isna().sum().sort_values(ascending=False).index[0]

In [134]:
# 테스트를 위한 샘플 데이터 생성
data = 도입부실[na_col].dropna()

In [135]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")


검정 통계량: 0.11454951763153076, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [136]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")


검정 통계량: 577.9172232548453
유의수준 15.0%: 임계값 = 0.575
유의수준 10.0%: 임계값 = 0.655
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.916
유의수준 1.0%: 임계값 = 1.09
데이터는 정규 분포를 따르지 않습니다.


In [137]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")


검정 통계량: 0.5023366544630479, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


fillna median

In [138]:
도입부실[na_col].fillna(data.median(), inplace=True)

# 성장정상

In [139]:
성장정상 = unlisted_X_train[(unlisted_X_train['성장기'] == 1) & (unlisted_X_train['부실판단'] == 0)]

결측치확인

In [140]:
성장정상.isna().sum().sort_values(ascending=False)

쭈피처                  1
회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

정규성 확인

In [141]:
na_col = 성장정상.isna().sum().sort_values(ascending=False).index[0]
# max_value = unlisted_X_train[inf_col].replace(np.inf, np.nan).max()
# unlisted_X_train[inf_col] = unlisted_X_train[inf_col].replace(np.inf, max_value)

In [142]:
data = 성장정상[na_col].dropna()

In [143]:
data.isna().sum()

0

In [144]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")


검정 통계량: 0.47999364137649536, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [145]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")


검정 통계량: 259.37435566781005
유의수준 15.0%: 임계값 = 0.575
유의수준 10.0%: 임계값 = 0.655
유의수준 5.0%: 임계값 = 0.786
유의수준 2.5%: 임계값 = 0.916
유의수준 1.0%: 임계값 = 1.09
데이터는 정규 분포를 따르지 않습니다.


In [146]:
from scipy.stats import kstest, norm


# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.5031569240307271, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


fillna median

In [147]:
성장정상[na_col].fillna(data.median(), inplace=True)

# 성장부실

In [148]:
성장부실 = unlisted_X_train[(unlisted_X_train['성장기'] == 1) & (unlisted_X_train['부실판단'] == 1)]

결측치 확인

In [149]:
성장부실.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

inf 확인

In [150]:
성장부실.isin([np.inf]).sum().sort_values(ascending=False)

총자산증가율               1
회사명                  0
매출액증가율               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
유동자산증가율              0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율   

In [151]:
inf_col = 성장부실.isin([np.inf]).sum().sort_values(ascending=False).index[0]
max_value = 성장부실[inf_col].replace(np.inf, np.nan).max()
성장부실[inf_col] = 성장부실[inf_col].replace(np.inf, max_value)

In [152]:
성장부실.isin([np.inf]).sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

# 성숙정상

In [153]:
성숙정상 = unlisted_X_train[(unlisted_X_train['성숙기'] == 1) & (unlisted_X_train['부실판단'] == 0)]

성숙정상 결측치 확인

In [154]:
성숙정상.isna().sum().sort_values(ascending=False)

쭈피처                  3
회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

정규성 확인

In [155]:
na_col = 성숙정상.isna().sum().sort_values(ascending=False).index[0]

In [156]:
len(성숙정상)

4660

In [157]:
# 테스트를 위한 샘플 데이터 생성
data = 성숙정상[na_col].dropna()

In [158]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.21134382486343384, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [159]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 823.7957742834269
유의수준 15.0%: 임계값 = 0.576
유의수준 10.0%: 임계값 = 0.655
유의수준 5.0%: 임계값 = 0.786
유의수준 2.5%: 임계값 = 0.917
유의수준 1.0%: 임계값 = 1.091
데이터는 정규 분포를 따르지 않습니다.


In [160]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.5011189771803239, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


fillna median

In [161]:
성숙정상.fillna(data.median(), inplace=True)

# 성숙부실

In [162]:
성숙부실 = unlisted_X_train[(unlisted_X_train['성숙기'] == 1) & (unlisted_X_train['부실판단'] == 1)]

성숙부실 결측치 확인

In [163]:
성숙부실.isna().sum().sort_values(ascending=False)

쭈피처                  6
재무레버리지               1
회사명                  0
경영자본회전률              0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
비유동자산회전률             0
총자본회전률               0
매출액증가율               0
유동자산회전률              0
영업년수                 0
영업레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

정규성 확인

In [164]:
na_col0 = 성숙부실.isna().sum().sort_values(ascending=False).index[0]
na_col1 = 성숙부실.isna().sum().sort_values(ascending=False).index[1]
# 테스트를 위한 샘플 데이터 생성
data0 = 성숙부실[na_col0].dropna()
data1 = 성숙부실[na_col1].dropna()

In [165]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.2980051636695862, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [166]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data1)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.3310622572898865, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [167]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 137.06956458237323
유의수준 15.0%: 임계값 = 0.573
유의수준 10.0%: 임계값 = 0.653
유의수준 5.0%: 임계값 = 0.784
유의수준 2.5%: 임계값 = 0.914
유의수준 1.0%: 임계값 = 1.087
데이터는 정규 분포를 따르지 않습니다.


In [168]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data1)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 185.93136271858157
유의수준 15.0%: 임계값 = 0.573
유의수준 10.0%: 임계값 = 0.653
유의수준 5.0%: 임계값 = 0.784
유의수준 2.5%: 임계값 = 0.914
유의수준 1.0%: 임계값 = 1.087
데이터는 정규 분포를 따르지 않습니다.


In [169]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.500046345399094, p-value: 5.198997032739048e-210
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [170]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data1, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.14866872064673958, p-value: 4.8506857072186544e-18
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


fillna median

In [171]:
성숙부실[na_col0].fillna(data0.median(), inplace=True)

In [172]:
성숙부실[na_col1].fillna(data1.median(), inplace=True)

# 쇠퇴정상

In [173]:
쇠퇴정상 = unlisted_X_train[(unlisted_X_train['쇠퇴기'] == 1) & (unlisted_X_train['부실판단'] == 0)]

결측치 확인

In [174]:
쇠퇴정상.isna().sum().sort_values(ascending=False)

영업재무현금비율             124
쭈피처                    1
회사명                    0
경영자본회전률                0
부가가치(백만원)              0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
유형자산회전율                0
비유동자산회전률               0
총자본회전률                 0
매출액증가율                 0
유동자산회전률                0
영업년수                   0
영업레버리지                 0
재무레버리지                 0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
유형자산증가율                0
비유동자산증가율               0
거래소코드                  0
현금비율                   0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0


정규성 확인

In [175]:
na_col0 = 쇠퇴정상.isna().sum().sort_values(ascending=False).index[0]
na_col1 = 쇠퇴정상.isna().sum().sort_values(ascending=False).index[1]
# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴정상[na_col0].dropna()
data1 = 쇠퇴정상[na_col1].dropna()

In [176]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.01838862895965576, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [177]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data1)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.14899367094039917, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [178]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 503.70699486728176
유의수준 15.0%: 임계값 = 0.574
유의수준 10.0%: 임계값 = 0.654
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.915
유의수준 1.0%: 임계값 = 1.089
데이터는 정규 분포를 따르지 않습니다.


In [179]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data1)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 376.5399996455867
유의수준 15.0%: 임계값 = 0.574
유의수준 10.0%: 임계값 = 0.654
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.916
유의수준 1.0%: 임계값 = 1.089
데이터는 정규 분포를 따르지 않습니다.


In [180]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.1679789483977252, p-value: 6.862637710098656e-34
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [181]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data1, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.501925515916277, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


쇠퇴정상 fillna median

In [182]:
쇠퇴정상[na_col0].fillna(data0.median(), inplace=True)

In [183]:
쇠퇴정상[na_col1].fillna(data1.median(), inplace=True)

# 쇠퇴부실

In [184]:
쇠퇴부실 = unlisted_X_train[(unlisted_X_train['쇠퇴기'] == 1) & (unlisted_X_train['부실판단'] == 1)]

결측치 확인

In [185]:
쇠퇴부실.isna().sum().sort_values(ascending=False)

영업재무현금비율             106
쭈피처                    1
회사명                    0
경영자본회전률                0
부가가치(백만원)              0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
유형자산회전율                0
비유동자산회전률               0
총자본회전률                 0
매출액증가율                 0
유동자산회전률                0
영업년수                   0
영업레버리지                 0
재무레버리지                 0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
유형자산증가율                0
비유동자산증가율               0
거래소코드                  0
현금비율                   0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0


정규성 확인

In [186]:
na_col0 = 쇠퇴부실.isna().sum().sort_values(ascending=False).index[0]
na_col1 = 쇠퇴부실.isna().sum().sort_values(ascending=False).index[1]
# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴부실[na_col0].dropna()
data1 = 쇠퇴부실[na_col1].dropna()

In [187]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.01935398578643799, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [188]:
from scipy.stats import shapiro

# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data1)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.26387739181518555, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [189]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 666.3560402047547
유의수준 15.0%: 임계값 = 0.575
유의수준 10.0%: 임계값 = 0.655
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.916
유의수준 1.0%: 임계값 = 1.09
데이터는 정규 분포를 따르지 않습니다.


In [190]:
from scipy.stats import anderson

# 안더슨-달링 검정 수행
result = anderson(data1)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 433.5936057408753
유의수준 15.0%: 임계값 = 0.575
유의수준 10.0%: 임계값 = 0.655
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.916
유의수준 1.0%: 임계값 = 1.09
데이터는 정규 분포를 따르지 않습니다.


In [191]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.19980988647451559, p-value: 4.839024161896977e-63
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [192]:
from scipy.stats import kstest, norm

# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data1, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.5031702699278638, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


median

In [193]:
쇠퇴부실[na_col0].fillna(data0.median(), inplace=True)

In [194]:
쇠퇴부실[na_col1].fillna(data1.median(), inplace=True)

# to_csv

train set concat

In [195]:
filled_train_data = pd.concat([도입부실, 도입정상, 성장부실, 성장정상, 성숙부실, 성숙정상, 쇠퇴부실, 쇠퇴정상], axis=0, ignore_index=True)

In [196]:
filled_train_data.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [197]:
len(filled_train_data)

15138

In [198]:
filled_train_data.to_csv('../data/datasets/unlisted_filled_train_data.csv', index=False)

test set

In [199]:
unlisted_X_test['부실판단'] = unlisted_y_test
unlisted_X_test['부실판단']

19945   0.000
6034    1.000
12336   0.000
19337   0.000
3974    0.000
         ... 
20311   1.000
9086    0.000
9694    0.000
10691   1.000
13411   1.000
Name: 부실판단, Length: 5569, dtype: float64

In [200]:
unlisted_X_test.to_csv('../data/datasets/unlisted_test_data.csv', index=False)

In [201]:
unlisted_X_test

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
19945,인지솔라주식회사,87600,2019,15.910,16.110,10.380,69.180,20.290,7.920,187.760,...,0.161,1.871,12.022,-0.793,2.408,0,0,1,0,0.000
6034,(주)케이엔에프인터내셔날,72390,2020,-23.920,-46.320,-21.590,-12.250,105.430,75.350,926.090,...,0.412,-1.397,0.972,-0.777,3.136,0,0,0,1,1.000
12336,경동산업주식회사,10708,2022,6.820,14.720,2.680,1.690,250.570,104.230,64.650,...,0.129,2.443,2.030,-0.880,5.588,0,0,1,0,0.000
19337,(주)장충동왕족발,53614,2022,7.550,10.030,4.640,5.440,64.220,45.060,159.340,...,0.346,11.815,2.035,-1.601,3.127,0,0,1,0,0.000
3974,(주)한강티에스,94815,2022,-8.520,0.000,-6.450,-1.240,171.520,171.520,312.100,...,1.009,-116.667,2.217,-1.019,46.686,0,0,0,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20311,조일산업(주),70253,2019,-11.470,-16.560,-7.770,-10.310,83.470,82.450,210.020,...,0.120,-86.965,-0.062,-4.403,-1.000,0,0,0,1,1.000
9086,디앤에이중공업(주),56937,2022,64.790,0.000,14.160,11.660,376.750,293.070,27.960,...,0.140,1.755,0.906,-16.095,-1.849,0,1,0,0,0.000
9694,(주)분더바움,81601,2020,910.550,9831.840,304.530,55.910,268.100,240.150,50.250,...,0.326,5.194,1.226,-7.998,-1.144,0,1,0,0,0.000
10691,빅스포주식회사,50162,2022,-3.970,-4.310,-2.550,-9.910,29.760,26.890,178.580,...,0.426,-1.137,0.781,-0.933,0.971,0,0,0,1,1.000
